In [ ]:
# We need PyTorch and GPyTorch
import torch
from torch import Tensor
import gpytorch
# Gaussian processes really don't work well without 64-bit precision
torch.set_default_dtype(torch.float64)
# For reproducibility:
torch.random.manual_seed(12345)

from norm_constrain import Norm2ConstrainedContainer_SE, Norm2ConstrainedContainer_rational, Norm2ConstrainedContainer_ConvexCombination

# And a few more players
import numpy as np
from scipy.integrate import quad
from matplotlib import pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
# Import the data, check normalization, which should be 2 * (2\pi**2)

with open('nk_afdmc.dat', 'r') as fd:
    lines = fd.readlines()

x = [] ; y = [] ; err = []
for line in lines:
    buf = line.split()
    x.append(float(buf[0]))
    y.append(float(buf[1]))
    err.append(float(buf[2]))
x = np.array(x) ; y = np.array(y) ; err = np.array(err)

dx = x[1]-x[0]
ig = (y * x**2).sum() *dx / (2 * np.pi**2)
print(ig)


In [ ]:
# Subsample training data
subrate = 10
train_x = torch.tensor(x[::subrate])
train_y = torch.tensor(y[::subrate])
train_err = torch.tensor(err[::subrate])**2


# Normalization value
nval = 4 * np.pi**2

# Create the GPyTorch model
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, nval):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        self.nm0 = Norm2ConstrainedContainer_SE(norm_val=nval)
        self.nm0.sigma = 2.947 ; self.nm0.gamma = 0.603 ; self.nm0.A = 0.752
        self.nm1 = Norm2ConstrainedContainer_rational(norm_val=nval)
        self.nm1.alpha = 1.687 ; self.nm1.p = 10.155 ; self.nm1.A = 1.400
        self.nm2 = Norm2ConstrainedContainer_SE(norm_val=nval)
        self.mod = Norm2ConstrainedContainer_ConvexCombination(norm_val=nval, kernels=(self.nm0, self.nm1,self.nm2))
        self.mod.compositions = torch.tensor([0.003, 0.996])
        # Using the normalization-constrained model
        self.covar_module = self.mod.covar_module
        self.mean_module = self.mod.mean_module
        
 
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train_err)

model = GPRegressionModel(train_x, train_y, likelihood, nval)



In [ ]:
# Training
training_iter = 1000

model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f\nCompositions: %s\nSE: sigma: %.3f  gamma: %.3f  A: %.3f\nRational 1: alpha: %.3f  p: %.3f  A: %.3f\nSE 2: sigma: %.3f  gamma: %.3f  A: %.3f\n' % (
        i + 1, training_iter, loss.item(),
        model.mod.full_compositions.tolist(),
        model.mod.kernels[0].sigma.item(),
        model.mod.kernels[0].gamma.item(),
        model.mod.kernels[0].A.item(),
        model.mod.kernels[1].alpha.item(),
        model.mod.kernels[1].p.item(),
        model.mod.kernels[1].A.item(),
        model.mod.kernels[2].sigma.item(),
        model.mod.kernels[2].gamma.item(),
        model.mod.kernels[2].A.item()
    ))
    optimizer.step()

In [ ]:
print("Final state:")
print('Loss: %.3f\nCompositions: %s\nSE: sigma: %.3f  gamma: %.3f  A: %.3f\nRational 1: alpha: %.3f  p: %.3f  A: %.3f\nSE 2: sigma: %.3f  gamma: %.3f  A: %.3f\n' % (
        loss.item(),
        model.mod.full_compositions.tolist(),
        model.mod.kernels[0].sigma.item(),
        model.mod.kernels[0].gamma.item(),
        model.mod.kernels[0].A.item(),
        model.mod.kernels[1].alpha.item(),
        model.mod.kernels[1].p.item(),
        model.mod.kernels[1].A.item(),
        model.mod.kernels[2].sigma.item(),
        model.mod.kernels[2].gamma.item(),
        model.mod.kernels[2].A.item()
    ))

In [ ]:
#Prediction time
N_Pred = 1000
x_max = x[-1]*1.05
pred_x = torch.linspace(0.0, x_max, N_Pred)
ind_t = np.arange(x.shape[0],dtype=int)
ind_t = ind_t[ind_t % subrate != 0]
test_x = x[ind_t]
test_y = y[ind_t]
test_err = err[ind_t]

tx = train_x.numpy().flatten()
ty = train_y.detach().numpy()
terr = train_err.sqrt().numpy()

model.eval()
likelihood.eval()
with torch.no_grad():
#    preds = likelihood(model(pred_x))
    preds = model(pred_x)

pred_x = pred_x.numpy()
lower, upper = preds.confidence_region() # 2-sigma confidence region
pm = preds.mean.numpy()
upp = upper.numpy()
low = lower.numpy()

def plotit(filename, xmin=None, xmax=None, ymin=None, ymax=None):

    fig1 = plt.figure()
    fig1.set_figwidth(16.0)
    fig1.set_figheight(8.0)
    ax1 = fig1.add_subplot(1,2,1)

    ax1.plot(pred_x, pm, "r-",label="Mean prediction")
    #ax1.fill_between(pred_x, low, upp, alpha = 0.3, label=r"2-$\sigma$ uncertainty")

    ax1.errorbar(test_x, test_y, yerr=test_err, ecolor='b', marker='o', ms=8, mfc='blue', mec='blue', label="Held out data",ls='None')
    ax1.errorbar(tx,ty, yerr=terr, ecolor='y', marker='v', ms=12, mfc='y', mec='y', ls='None', label="Training data")
    ax1.legend()

    ax1.tick_params(labelsize=20)
    ax1.set_xlabel('X', size=20)
    ax1.set_ylabel(r'Y', size=20)
    ax1.set_ylim(ymin=ymin, ymax=ymax)
    ax1.set_xlim(xmin=xmin, xmax=xmax)

    ax2 = fig1.add_subplot(1,2,2)

    ax2.plot(pred_x, pred_x**2 *pm, "r-",label="Mean prediction")
    #ax2.fill_between(pred_x, pred_x**2 * low, pred_x**2 * upp, alpha = 0.3, label=r"2-$\sigma$ uncertainty")

    ax2.errorbar(test_x, test_x**2 * test_y, yerr=test_x**2 * test_err, ecolor='b', marker='o', ms=8, mfc='blue', mec='blue', label="Held out data",ls='None')
    ax2.errorbar(tx,tx**2 * ty, yerr=tx**2 * terr, ecolor='y', marker='v', ms=12, mfc='y', mec="y", ls='None', label="Training data")

    ax2.legend()

    ax2.tick_params(labelsize=20)
    ax2.set_xlabel('X', size=20)
    ax2.set_ylabel(r'Y$\times$X$^2$', size=20)

    ax2.set_ylim(ymin=ymin, ymax=ymax)
    ax2.set_xlim(xmin=xmin, xmax=xmax)

    plt.savefig(filename, format="png")

plotit("gp_momentum_emulator.png")

In [ ]:
# Zoom in
plotit("gp_momentum_emulator_zoom.png", xmax=2.5)

In [ ]:
# Check mean function normalization
def integrand(x):
    z = torch.tensor([x])
    with torch.no_grad():
        pred = model(z) / (2 * np.pi**2)
    mean = pred.mean * z**2
    return mean.detach().numpy()

quad_res = quad(integrand, 0.0, np.inf)
print("Quadrature integral = %.9f, error = %10.3E" % quad_res)

# Get the mse of held-out values
Test_x = torch.tensor(test_x) ; Test_y = torch.tensor(test_y)
with torch.no_grad():
    pred = model(Test_x)
mean = pred.mean
mse = ((pred.mean - test_y)**2).mean().sqrt()
nrm = (pred.mean**2).mean().sqrt()
mse = mse / nrm
print("Normalized MSE of held-out data = %12.5E" % mse)

# Largest relative error
relerr = (pred.mean - test_y).abs() / pred.mean.abs()
relerr_max, ind = relerr.max(dim=0)
print("Largest relative error = %12.5E at x = %12.5E" % (relerr_max, Test_x[ind]))
print("Mean relative error = %12.5E" % relerr.mean())

# Restrict to x < xmax, repeat error analysis
xmax = 2.5
Tx2 = Test_x[Test_x < xmax] ; Ty2 = Test_y[Test_x<xmax] ; mn2 = pred.mean[Test_x<xmax]
mse = ((mn2 - Ty2)**2).mean().sqrt()
nrm = (mn2**2).mean().sqrt()
mse = mse / nrm
relerr = (mn2 - Ty2).abs() / mn2.abs()
relerr_max, ind = relerr.max(dim=0)
print("\nRestricted to x < %f:" % xmax)
print("Normalized MSE of held-out data = %12.5E" % mse)
print("Largest relative error = %12.5E at x = %12.5E" % (relerr_max, Tx2[ind]))
print("Mean relative error = %12.5E" % relerr.mean())

